In [1]:
import sys
import os
import subprocess
import time
# Need to clone FinGPT github
from multisource_retrieval.external_LLMs import external_LLMs2
import pandas as pd
import threading
import time
import pandas as pd
from datetime import datetime
import ast

In [2]:
def run_with_timeout(func, timeout):
    exception = [None]

    def worker():
        try:
            func()
        except Exception as e:
            exception[0] = e

    thread = threading.Thread(target=worker)
    thread.start()
    thread.join(timeout=timeout)

    if thread.is_alive():
        thread.join()
        raise Exception("Function execution exceeded the timeout limit")

    if exception[0]:
        raise exception[0]

In [3]:
sentence_column = 'text'

In [17]:

def classify_rows():
    global df, row_index, file_path
    counter = 0
    output_file_path = os.path.splitext('')[0] + "_classified.csv"
    
    MAX_RETRIES = 5
    DELAY_BETWEEN_RETRIES = 10  # 10 seconds

    for current_index, row in df.iloc[row_index:].iterrows():
        target_sentence = row[sentence_column]
        
        retries = 0
        while retries < MAX_RETRIES:
            try:
                classification_response = external_LLMs2.extract_classification(target_sentence, classification_prompt)
        
                classification_data = classification_response.split('(')[1].split(')')[0].split(', ')
                ticker = classification_data[0].strip()
                df.at[current_index, 'ticker'] = ticker

                sentiment_score = float(classification_data[1].strip()) 
                df.at[current_index, 'sentiment_score'] = sentiment_score
                break
            except Exception as e:
                print(f"Exception during classification: {e}")
                retries += 1
                time.sleep(DELAY_BETWEEN_RETRIES)
        else:  # This will execute if the while loop exhausts all retries without a break
            print(f"Max retries reached for row {current_index}. Moving to the next row.")
 
        time.sleep(3)
        counter += 1

        if counter % 5 == 0:
            df.to_csv(output_file_path, index=False)

json dataException during classification: Socket operation on non-socket


json data {'id': 'chatcmpl-8dZxh6J0P5vr2XNbNv8HykcE1ijaE', 'object': 'chat.completion', 'created': 1704443721, 'model': 'gpt-4-1106-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '(DOT, 0.00)'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 246, 'completion_tokens': 8, 'total_tokens': 254}, 'system_fingerprint': 'fp_c6efb4aa39'}
Classification response: (DOT, 0.00)


In [5]:
df = pd.read_csv('')
column_names = df.columns.tolist()

In [12]:
row_index = 0

In [15]:
default_classification_prompt = ". For the crypto tweets above, please do the following: \
Determine the sentiment scores for the following cryptocurrency tickers. \
The range of the sentiment score is from -1.00 (most negative) to 1.00 (most positive). \
Selectable ticker list: BTC, ETH, SOL, BNB, XRP, ADA, AVAX, DOGE, DOT, TRX, LINK, MATIC, TON, SHIB, LTC, BCH, ICP, ATOM, UNI. \
Must provide the output in the following tuple format: (ticker, sentiment score).\
Additonal text must not be allowed. Only output."
classification_prompt = default_classification_prompt

json dataException during classification: Socket operation on non-socket
Extracting classification for RT @ClaraVanStaden: https://t.co/FXtMfK0YZO
Snowbridge has been merged into the Asset Hub and Bridge Hub Rococo runtimes. Deployment soon.…
Sending request to https://api.openai.com/v1/chat/completions with payload {'model': 'gpt-4-1106-preview', 'messages': [{'role': 'system', 'content': 'You are a crypto analyst. You have two important tasks to deal with.                 1. Extract ticker for the tweets raw data. 2. Respond with a sentiment score out of the following options below for a classification task.                 Output format: (ticker, sentiment score).                 Ensure the format of the output is correct.'}, {'role': 'user', 'content': 'We have the following crypto tweets raw data: "RT @ClaraVanStaden: https://t.co/FXtMfK0YZO\nSnowbridge has been merged into the Asset Hub and Bridge Hub Rococo runtimes. Deployment soon.…". For the crypto tweets above, please do the

In [16]:
run_with_timeout(classify_rows, 3000)

Extracting classification forException during classification: Socket operation on non-socket
Extracting classification for My annual Polkadot Roundup is out - enjoy! https://t.co/Cwyydmx371
Sending request to https://api.openai.com/v1/chat/completions with payload {'model': 'gpt-4-1106-preview', 'messages': [{'role': 'system', 'content': 'You are a crypto analyst. You have two important tasks to deal with.                 1. Extract ticker for the tweets raw data. 2. Respond with a sentiment score out of the following options below for a classification task.                 Output format: (ticker, sentiment score).                 Ensure the format of the output is correct.'}, {'role': 'user', 'content': 'We have the following crypto tweets raw data: "My annual Polkadot Roundup is out - enjoy! https://t.co/Cwyydmx371". For the crypto tweets above, please do the following: Determine the sentiment scores for the following cryptocurrency tickers. The range of the sentiment score is from -

KeyboardInterrupt: 